In [ ]:
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers
!pip -q install langchain
!pip -q install gradio
!pip -q install peft chromadb
!pip -q install unstructured
!pip install -q sentence_transformers
!pip -q install pypdf
!pip install -U langchain-community
!nvidia-smi

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Fri Jul 12 04:24:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                      

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False)
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config,device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!mkdir docs

mkdir: cannot create directory ‘docs’: File exists


In [ ]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

os.makedirs('/content/docs', exist_ok=True)
pdf_files = [f for f in os.listdir('/content/docs') if f.endswith('.pdf')]
pages = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
)

for pdf_file in pdf_files:
    loader = PyPDFLoader(os.path.join('/content/docs', pdf_file))
    pages.extend(loader.load_and_split(text_splitter))

db = Chroma.from_documents(pages, HuggingFaceEmbeddings(), persist_directory='/content/db')


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
db = Chroma.from_documents(pages, HuggingFaceEmbeddings(), persist_directory='/content/db')

In [ ]:
import json
import textwrap
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
import gradio as gr
import random
import time
from transformers import pipeline

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. DO NOT REITERATE THE GIVEN INPUT TEXT. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

instruction = "Given the context that has been provided. \n {context}, Answer the following question - \n{question}"

system_prompt = """You will be given a context to answer from. Be precise in your answers wherever possible.
In case you are sure you don't know the answer then you say that based on the context you don't know the answer.
In all other instances you provide an answer to the best of your capability. Cite urls when you can access them related to the context. Do not reiterste the given input text. """

get_prompt(instruction, system_prompt)

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=0,
    return_messages=False
)

retriever = db.as_retriever()

def create_pipeline(max_new_tokens=512, temperature=0.1):
    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens=max_new_tokens,
                    temperature=temperature)
    return pipe

class ContextualQA:
    def __init__(self, memory, prompt, retriever, max_new_tokens=512, temperature=0.1):
        self.memory = memory
        self.prompt = prompt
        self.retriever = retriever
        self.max_new_tokens = max_new_tokens
        self.temperature = temperature
        self.qa = self.create_chat_bot()

    def create_chat_bot(self):
        hf_pipe = create_pipeline(self.max_new_tokens, self.temperature)
        llm = HuggingFacePipeline(pipeline=hf_pipe)
        qa = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=self.retriever,
            memory=self.memory,
            combine_docs_chain_kwargs={"prompt": self.prompt}
        )
        return qa

    def update_parameters(self, max_new_tokens=None, temperature=None):
        if max_new_tokens is not None:
            self.max_new_tokens = max_new_tokens
        if temperature is not None:
            self.temperature = temperature
        self.qa = self.create_chat_bot()

contextual_qa = ContextualQA(memory=memory, prompt=prompt, retriever=retriever)
bot = contextual_qa.qa

def clear_llm_memory():
    bot.memory.clear()

def update_prompt(sys_prompt):
    if sys_prompt == "":
        sys_prompt = system_prompt
    template = get_prompt(instruction, sys_prompt)
    new_prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    contextual_qa.prompt = new_prompt
    contextual_qa.qa.combine_docs_chain.llm_chain.prompt = new_prompt

def extract_response(response):
    parts = response.split(E_INST)
    if len(parts) > 1:
        return parts[1].strip()
    return response

def update_parameters(max_new_tokens, temperature):
    contextual_qa.update_parameters(max_new_tokens, temperature)

with gr.Blocks(css="""
 .gradio-container {background-color:#f2fafa; font-family: Arial, sans-serif;}
 .settings-header {border: 0.5px solid #b8b6b6; padding: 10px; background-color: #f0f2f2; margin-bottom: 5px; border-radius: 7px;text-shadow: 1px 1px 1px rgba(0, 0, 0, 0.1);}
 .chat-header {border: 0.5px solid #b8b6b6; padding: 10px; background-color: #a6ddff; margin-bottom: 5px; border-radius: 7px; font-weight: bold; color: white; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.5);} /* Bold font, white text, and shadow for decoration */
 .chatbot-container.message {background-color: #dee2e3; font-size: 14px; max-width: 100%;}  /* Smaller font size and increased width */
 .chat-header {margin-bottom: 2px;} /* Reduced bottom margin for the chat header */
""") as demo:
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("<h2 class='settings-header' style='text-align: center; color: #575555;'>Settings</h2>")
            update_sys_prompt = gr.Textbox(label="Update System Prompt", placeholder="Enter new system prompt here...")
            clear_memory = gr.Button(value="Clear LLM Memory", elem_id="clear-btn")
            gr.Markdown("#### Temperature")
            temperature_slider = gr.Slider(minimum=0.0, maximum=1.0, step=0.1, value=0.1, label="Lower values for conventional responses, higher for creative exploration.")
            gr.Markdown("#### Max New Tokens")
            token_slider = gr.Slider(minimum=1, maximum=1024, step=1, value=512, label="Set the maximum number of tokens for responses.")
        with gr.Column(scale=5):
            gr.Markdown("<h2 class='chat-header' style='text-align: center;'>INCOIS Chatbot</h2>")
            chatbot = gr.Chatbot(label="Contextual QA", height=600, elem_classes="chatbot-container")
            msg = gr.Textbox(label="Question", placeholder="Ask your question here...")
            clear = gr.ClearButton([msg, chatbot])


    def respond(message, chat_history, max_new_tokens, temperature):
        update_parameters(max_new_tokens, temperature)
        response = bot({"question": message})['answer']
        bot_message = extract_response(response)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, inputs=[msg, chatbot, token_slider, temperature_slider], outputs=[msg, chatbot])
    clear_memory.click(clear_llm_memory)
    update_sys_prompt.submit(update_prompt, inputs=update_sys_prompt)

demo.launch(share=True, debug=True)


[INST]<<SYS>>
You will be given a context to answer from. Be precise in your answers wherever possible.The maximum number of tokens for responses is 50.
In case you are sure you don't know the answer then you say that based on the context you don't know the answer.
In all other instances you provide an answer to the best of your capability. Cite urls when you can access them related to the context. DO NOT REITERATE THE GIVEN INPUT TEXT. 
<</SYS>>

Given the context that has been provided. 
 {context}, Answer the following question - 
{question}[/INST]
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ef0b0db6ec9ecca3ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ef0b0db6ec9ecca3ad.gradio.live
